<a href="https://colab.research.google.com/github/itsZENR/Reverse-a-cellular-automaton-using-a-genetic-algorithm/blob/main/Reverse_a_cellular_automaton_using_a_genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [95]:
import numpy as np
import random
from deap import algorithms, base, creator, tools

# Привило игры жизнь

In [7]:
def rule(grid, i, j):
    # получаем размер сетки
    rows, cols = len(grid), len(grid[0])
    
    # вычисляем сумму значений соседних клеток
    neighbors_sum = (
        grid[(i-1)%rows][(j-1)%cols] + grid[(i-1)%rows][j] + grid[(i-1)%rows][(j+1)%cols] +
        grid[i][(j-1)%cols] + grid[i][(j+1)%cols] +
        grid[(i+1)%rows][(j-1)%cols] + grid[(i+1)%rows][j] + grid[(i+1)%rows][(j+1)%cols]
    )
    
    # определяем новое состояние клетки на основе ее текущего состояния и суммы значений соседей
    if grid[i][j] == 1:
        if neighbors_sum < 2 or neighbors_sum > 3:
            return 0
        else:
            return 1
    else:
        if neighbors_sum == 3:
            return 1
        else:
            return 0

# Добавить рамку из нулей

In [6]:
def add_border(arr):
    rows, cols = np.shape(arr)
    new_arr = np.zeros((rows+2, cols+2))
    new_arr[1:-1, 1:-1] = arr
    return new_arr

# Убрать рамку из нулей

In [5]:
def remove_border(matrix):
    # Получаем размеры матрицы
    rows, cols = len(matrix), len(matrix[0])
    
    # Создаем новую матрицу размером на 2 строки и 2 столбца меньше
    new_matrix = [[0] * (cols - 2) for _ in range(rows - 2)]
    
    # Копируем значения из старой матрицы в новую без учета рамки
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            new_matrix[i - 1][j - 1] = matrix[i][j]
    
    return np.array(new_matrix)

# Таблицы

In [92]:
grid = [[1,1,1]
       ,[1,0,1]
       ,[1,1,1]]

In [27]:
grid = [[0,0,0,0,0]
       ,[0,1,1,1,0]
       ,[0,1,1,1,0]
       ,[0,1,1,1,0]
       ,[0,0,0,0,0]]

In [24]:
grid = [[1,1]
       ,[1,1]]

In [74]:
grid = [[1]]

In [63]:
import numpy as np

# создаем два двумерных массива
a = np.array([[1, 0]
             ,[1, 0]])

b = np.array([[1, 1]
             ,[1, 0]])

# находим индексы элементов, которые находятся в одинаковых позициях в массивах a и b
indices = np.where(a == b)

# выводим значения и индексы элементов, которые находятся в одинаковых позициях
for i in range(len(indices[0])):
    # print("Value:", a[indices[0][i], indices[1][i]], "Index:", (indices[0][i], indices[1][i]))
    print("Index:", (indices[0][i], indices[1][i]))

len(indices[0])

Index: (0, 0)
Index: (1, 0)
Index: (1, 1)


3

# Итерация на шаг вперед

In [2]:
def iter(grid):
  border_grid = add_border(grid)
  n = len(border_grid)
  m = len(border_grid[0])

  new_border_grid = np.zeros((n, m))
  for i in range(n):
    for j in range(m):
      new_border_grid[i][j] = rule(border_grid, i, j)

  return remove_border(new_border_grid)

# Перебор (Просто показывает все варианты и показывает их следующую итерацию)

In [ ]:
n = len(grid)+2
m = len(grid[0])+2

k = n*m
    

def find(i = 0):
    if i == k:
        print('r')
        print(a.reshape(n, m))
        print()
        print('i')
        print(iter(a.reshape(n, m)))
        print()
        return
    a[i] = 0
    find(i + 1)
    a[i] = 1
    find(i + 1)


a = np.zeros(k)
find()

# Генетический алгоритм

In [11]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 KB 3.8 MB/s eta 0:00:00


In [101]:
# Создаем объект-класс FitnessMin для минимизации функции
creator.create("FitnessMin", base.Fitness, weights=(1.0,))

# Создаем объект-класс Individual, наследующийся от списка и имеющий атрибут fitness
creator.create("Individual", list, fitness=creator.FitnessMin)

n = len(grid)+2
m = len(grid[0])+2

# Задаем параметры алгоритма и создаем популяцию
pop_size = 1000
gen_size = n*m
cx_prob = 0.5
mut_prob = 0.2
pop = [creator.Individual([random.randint(0, 1) for _ in range(gen_size)]) for _ in range(pop_size)]

# Определяем функцию оценки приспособленности
def eval_one_max(individual):
    indices = np.where(add_border(grid) == iter(np.array(individual).reshape(n,m)))
    return len(indices[0]),

# Создаем объект-класс Toolbox и регистрируем необходимые функции
toolbox = base.Toolbox()
toolbox.register("evaluate", eval_one_max)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=mut_prob)
toolbox.register("select", tools.selTournament, tournsize=3)

# Запускаем генетический алгоритм
num_generations = 50
for gen in range(num_generations):
    offspring = algorithms.varAnd(pop, toolbox, cxpb=cx_prob, mutpb=mut_prob)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    pop = toolbox.select(offspring, k=len(pop))
    
# Получаем лучшее решение и его оценку приспособленности
best_ind = tools.selBest(pop, k=1)[0]
best_fitness = best_ind.fitness.values[0]

print("Best individual is ", best_ind)
print("Best fitness is ", best_fitness)

Best individual is  [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
Best fitness is  25.0
